# Excel Pivot Tables with win32com

This notebook demonstrates how to create **true Excel PivotTables** using win32com (pywin32).

**Requirements:**
- Windows OS
- Microsoft Excel installed
- Python packages: `pandas`, `pywin32`, `openpyxl`

**Advantages of win32com:**
- Creates real, interactive PivotTables (not static tables)
- Full Excel formatting and styling
- Users can modify pivots after generation
- Supports all Excel chart types

## 1. Import Required Libraries

In [8]:
import pandas as pd
import os
from pathlib import Path

try:
    import win32com.client as win32
    from win32com.client import constants as c
    WIN32COM_AVAILABLE = True
    print("✓ win32com imported successfully")
except ImportError:
    WIN32COM_AVAILABLE = False
    print("✗ pywin32 not installed. Run: pip install pywin32")

✓ win32com imported successfully


## 2. Create Sample Data

We'll create sample sales data with:
- Date range (100 days)
- 4 Regions (North, South, East, West)
- 4 Products (A, B, C, D)
- Sales and Quantity metrics

In [9]:
def create_sample_data():
    """Create sample sales data for demonstration"""
    data = {
        'Date': pd.date_range('2024-01-01', periods=100, freq='D'),
        'Region': ['North', 'South', 'East', 'West'] * 25,
        'Product': ['Product A', 'Product B', 'Product C', 'Product D'] * 25,
        'Sales': [100, 150, 200, 175, 120, 180, 210, 190] * 12 + [100, 150, 200, 175],
        'Quantity': [10, 15, 20, 18, 12, 16, 22, 19] * 12 + [10, 15, 20, 18]
    }
    return pd.DataFrame(data)

# Create and display sample data
df = create_sample_data()
print(f"Created {len(df)} rows of sample data")
df.head(10)

Created 100 rows of sample data


,Date,Region,Product,Sales,Quantity
0,2024-01-01,North,Product A,100,10
1,2024-01-02,South,Product B,150,15
2,2024-01-03,East,Product C,200,20
3,2024-01-04,West,Product D,175,18
4,2024-01-05,North,Product A,120,12
5,2024-01-06,South,Product B,180,16
6,2024-01-07,East,Product C,210,22
7,2024-01-08,West,Product D,190,19
8,2024-01-09,North,Product A,100,10
9,2024-01-10,South,Product B,150,15


## 3. Excel Constants Reference

When using win32com without type libraries, we use numeric constants:

| Constant | Value | Description |
|----------|-------|-------------|
| xlUp | -4162 | Navigate upward |
| xlDown | -4121 | Navigate downward |
| xlToLeft | -4159 | Navigate left |
| xlToRight | -4161 | Navigate right |
| xlSum | -4157 | Sum aggregation |
| xlAverage | -4106 | Average aggregation |
| xlRowField | 1 | Row field orientation |
| xlColumnField | 2 | Column field orientation |
| xlDatabase | 1 | Database source type |
| xlColumnClustered | 51 | Clustered column chart |
| xlPie | 5 | Pie chart |

## 4. Main Function: Create PivotTable with Charts

In [ ]:
def create_true_pivot_with_win32com(input_file=None, output_file='output_win32com.xlsx'):
    """
    Create Excel PivotTables and charts using win32com
    
    Args:
        input_file: Path to input Excel file (if None, uses sample data)
        output_file: Path to output Excel file
    """
    
    if not WIN32COM_AVAILABLE:
        print("ERROR: This function requires pywin32 and Excel to be installed.")
        return
    
    # Read or create data
    if input_file:
        df = pd.read_excel(input_file)
        print(f"Reading data from: {input_file}")
    else:
        df = create_sample_data()
        print("Using sample data (no input file provided)")
    
    # Save data to a temporary Excel file using pandas
    temp_file = 'temp_data_for_pivot.xlsx'
    df.to_excel(temp_file, sheet_name='Raw Data', index=False)
    print(f"✓ Saved temporary file: {temp_file}")
    
    # Convert to absolute path
    abs_output_path = str(Path(output_file).resolve())
    abs_temp_path = str(Path(temp_file).resolve())
    
    # Start Excel application using DispatchEx for more reliable COM automation
    # DispatchEx creates a new Excel instance and avoids COM cache issues
    print("Starting Excel...")
    excel = win32.DispatchEx('Excel.Application')
    excel.Visible = False  # Set to True if you want to see Excel working
    excel.DisplayAlerts = False
    
    try:
        # Open the workbook
        wb = excel.Workbooks.Open(abs_temp_path)
        ws_data = wb.Worksheets('Raw Data')
        print("✓ Opened workbook")
        
        # Get data range
        last_row = ws_data.Cells(ws_data.Rows.Count, 1).End(-4162).Row  # xlUp = -4162
        last_col = ws_data.Cells(1, ws_data.Columns.Count).End(-4159).Column  # xlToLeft = -4159
        data_range = ws_data.Range(ws_data.Cells(1, 1), ws_data.Cells(last_row, last_col))
        print(f"✓ Data range: {last_row} rows x {last_col} columns")
        
        # ============ Create First PivotTable: Sales by Region ============
        print("\nCreating first PivotTable: Sales by Region...")
        ws_pivot1 = wb.Worksheets.Add()
        ws_pivot1.Name = "Pivot - Sales by Region"
        
        # Create PivotCache
        pivot_cache = wb.PivotCaches().Create(
            SourceType=1,  # xlDatabase
            SourceData=data_range
        )
        
        # Create PivotTable
        pivot_table1 = pivot_cache.CreatePivotTable(
            TableDestination=ws_pivot1.Range("A3"),
            TableName="SalesByRegion"
        )
        
        # Configure PivotTable fields
        # Add Region to Row
        pivot_table1.PivotFields("Region").Orientation = 1  # xlRowField
        pivot_table1.PivotFields("Region").Position = 1
        
        # Add Product to Column
        pivot_table1.PivotFields("Product").Orientation = 2  # xlColumnField
        pivot_table1.PivotFields("Product").Position = 1
        
        # Add Sales to Values
        pivot_table1.AddDataField(
            pivot_table1.PivotFields("Sales"),
            "Sum of Sales",
            -4157  # xlSum
        )
        
        # Apply style
        pivot_table1.TableStyle2 = "PivotStyleMedium9"
        print("✓ PivotTable created")
        
        # Create a chart from the PivotTable
        print("Creating chart...")
        chart1 = ws_pivot1.Shapes.AddChart2(251, 51).Chart  # xlColumnClustered
        chart1.SetSourceData(pivot_table1.TableRange2)
        chart1.ChartType = 51  # xlColumnClustered
        chart1.HasTitle = True
        chart1.ChartTitle.Text = "Sales by Region and Product"
        
        # Position the chart
        chart_shape = chart1.Parent
        chart_shape.Top = ws_pivot1.Range("G3").Top
        chart_shape.Left = ws_pivot1.Range("G3").Left
        chart_shape.Width = 400
        chart_shape.Height = 300
        print("✓ Chart created")
        
        # ============ Create Second PivotTable: Product Statistics ============
        print("\nCreating second PivotTable: Product Stats...")
        ws_pivot2 = wb.Worksheets.Add()
        ws_pivot2.Name = "Pivot - Product Stats"
        
        pivot_cache2 = wb.PivotCaches().Create(
            SourceType=1,
            SourceData=data_range
        )
        
        pivot_table2 = pivot_cache2.CreatePivotTable(
            TableDestination=ws_pivot2.Range("A3"),
            TableName="ProductStats"
        )
        
        # Configure second PivotTable
        pivot_table2.PivotFields("Product").Orientation = 1  # xlRowField
        
        # Add multiple value fields
        pivot_table2.AddDataField(
            pivot_table2.PivotFields("Quantity"),
            "Sum of Quantity",
            -4157  # xlSum
        )
        
        pivot_table2.AddDataField(
            pivot_table2.PivotFields("Quantity"),
            "Average Quantity",
            -4106  # xlAverage
        )
        
        pivot_table2.TableStyle2 = "PivotStyleMedium2"
        print("✓ Second PivotTable created")
        
        # Create a pie chart for product distribution
        print("Creating pie chart...")
        chart2 = ws_pivot2.Shapes.AddChart2(201, 5).Chart  # xlPie
        chart2.SetSourceData(pivot_table2.TableRange2)
        chart2.ChartType = 5  # xlPie
        chart2.HasTitle = True
        chart2.ChartTitle.Text = "Quantity Distribution by Product"
        
        # Position the chart
        chart_shape2 = chart2.Parent
        chart_shape2.Top = ws_pivot2.Range("F3").Top
        chart_shape2.Left = ws_pivot2.Range("F3").Left
        chart_shape2.Width = 350
        chart_shape2.Height = 300
        
        # Add data labels to pie chart
        chart2.SeriesCollection(1).HasDataLabels = True
        chart2.SeriesCollection(1).DataLabels().ShowPercentage = True
        print("✓ Pie chart created")
        
        # Save as new file
        if os.path.exists(abs_output_path):
            os.remove(abs_output_path)
        
        wb.SaveAs(abs_output_path)
        wb.Close()
        print(f"\n✓ Saved Excel file: {output_file}")
        
    except Exception as e:
        print(f"✗ ERROR: {e}")
        raise
    
    finally:
        # Clean up
        excel.Quit()
        if os.path.exists(temp_file):
            os.remove(temp_file)
        print("✓ Cleaned up temporary files")

print("Function defined: create_true_pivot_with_win32com()")

Function defined: create_true_pivot_with_win32com()


## 5. Run: Create PivotTable from Sample Data

This will:
1. Create sample sales data
2. Generate two PivotTables with charts
3. Save to `output_win32com.xlsx`

In [11]:
# Create Excel file with PivotTables and charts
create_true_pivot_with_win32com()

print("\n" + "="*50)
print("DONE! Open 'output_win32com.xlsx' to see the results.")
print("The file contains:")
print("  - Summary sheet")
print("  - Raw Data sheet")
print("  - Pivot - Sales by Region (with chart)")
print("  - Pivot - Product Stats (with pie chart)")
print("="*50)

Using sample data (no input file provided)
✓ Saved temporary file: temp_data_for_pivot.xlsx
Starting Excel...
✓ Opened workbook
✓ Data range: 101 rows x 5 columns

Creating first PivotTable: Sales by Region...
✓ PivotTable created
Creating chart...
✓ Chart created

Creating second PivotTable: Product Stats...
✓ Second PivotTable created
Creating pie chart...
✓ Pie chart created

✓ Saved Excel file: output_win32com.xlsx
✓ Cleaned up temporary files

DONE! Open 'output_win32com.xlsx' to see the results.
The file contains:
  - Summary sheet
  - Raw Data sheet
  - Pivot - Sales by Region (with chart)
  - Pivot - Product Stats (with pie chart)


## 6. Alternative: Add PivotTable to Existing Excel File

In [12]:
def add_pivot_to_existing_file(input_file, output_file='updated_win32com.xlsx'):
    """
    Add PivotTable to an existing Excel file
    
    Args:
        input_file: Path to existing Excel file
        output_file: Path to save updated file
    """
    
    if not WIN32COM_AVAILABLE:
        print("ERROR: This function requires pywin32 and Excel to be installed.")
        return
    
    abs_input_path = str(Path(input_file).resolve())
    abs_output_path = str(Path(output_file).resolve())
    
    # Use DispatchEx for more reliable COM automation
    excel = win32.DispatchEx('Excel.Application')
    excel.Visible = False
    excel.DisplayAlerts = False
    
    try:
        wb = excel.Workbooks.Open(abs_input_path)
        ws_data = wb.Worksheets(1)  # First sheet
        
        # Get data range
        # -4162 (xlUp) - Always means "go up"
        # -4159 (xlToLeft) - Always means "go left"
        last_row = ws_data.Cells(ws_data.Rows.Count, 1).End(-4162).Row      
        last_col = ws_data.Cells(1, ws_data.Columns.Count).End(-4159).Column
        data_range = ws_data.Range(ws_data.Cells(1, 1), ws_data.Cells(last_row, last_col))
        
        # Create new sheet for pivot
        ws_pivot = wb.Worksheets.Add()
        ws_pivot.Name = "New PivotTable"
        
        # Create PivotTable
        pivot_cache = wb.PivotCaches().Create(
            SourceType=1,
            SourceData=data_range
        )
        
        pivot_table = pivot_cache.CreatePivotTable(
            TableDestination=ws_pivot.Range("A3"),
            TableName="NewPivot"
        )
        
        # Configure with first available fields
        fields = [pivot_table.PivotFields(i) for i in range(1, min(4, last_col + 1))]
        
        if len(fields) >= 2:
            fields[0].Orientation = 1  # Row field
            fields[1].Orientation = 4  # Data field
        
        wb.SaveAs(abs_output_path)
        wb.Close()
        
        print(f"✓ PivotTable added to existing file: {output_file}")
        
    except Exception as e:
        print(f"✗ ERROR: {e}")
        raise
    
    finally:
        excel.Quit()

print("Function defined: add_pivot_to_existing_file()")

Function defined: add_pivot_to_existing_file()


## 7. Example: Add Pivot to Existing File

Uncomment and run to add a PivotTable to your own Excel file:

In [13]:
# Example: Add pivot to an existing file
# add_pivot_to_existing_file('your_input_file.xlsx', 'updated_win32com.xlsx')

# Or use the output file we just created
# add_pivot_to_existing_file('output_win32com.xlsx', 'updated_win32com.xlsx')

print("Uncomment the code above to add a pivot to an existing file")

Uncomment the code above to add a pivot to an existing file


## 8. Troubleshooting

### Common Issues and Solutions:

**1. ModuleNotFoundError: No module named 'win32com'**
```bash
pip install pywin32
```

**2. ModuleNotFoundError: No module named 'openpyxl'**
```bash
pip install openpyxl pandas
```

**3. COM Error: Call was rejected by callee**
- Close all Excel instances
- Clear win32com cache:
```python
import win32com
import shutil
from pathlib import Path
cache_path = Path(win32com.__gen_path__)
shutil.rmtree(cache_path, ignore_errors=True)
```

**4. Excel not installed**
- win32com requires Excel to be installed on Windows
- For systems without Excel, use `xlsxwriter` or `openpyxl` instead

### Why DispatchEx?

We use `win32.DispatchEx()` instead of `win32.Dispatch()` because:
- Creates a new, independent Excel instance
- Avoids COM cache corruption issues
- More reliable across different environments
- Doesn't require running makepy manually